# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [1]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [1]:
# TODO: Import the necessary libs
# For example: 
import os

from lib.agents import Agent
from lib.llm import LLM
from lib.messages import UserMessage, SystemMessage, ToolMessage, AIMessage
from lib.state_machine import StateMachine, Step, EntryPoint, Termination, Run
from lib.tooling import ToolCall,tool
from dotenv import load_dotenv
from typing import List, Dict, Any, Optional, TypedDict
import chromadb
from chromadb.utils import embedding_functions
import json
from tavily import TavilyClient
from pydantic import BaseModel, Field

In [2]:
# TODO: Load environment variables
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [3]:
# TODO: Create retrieve_game tool
# It should use chroma client and collection you created
# chroma_client = chromadb.PersistentClient(path="chromadb")
# collection = chroma_client.get_collection("udaplay")
# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - query: a question about game industry. 
#
#    You'll receive results as list. Each element contains:
#    - Platform: like Game Boy, Playstation 5, Xbox 360...)
#    - Name: Name of the Game
#    - YearOfRelease: Year when that game was released for that platform
#    - Description: Additional details about the game

chroma_client = chromadb.PersistentClient(path="chromadb")
collection = chroma_client.get_collection("udaplay")

@tool
def retrieve_game(query: str) -> List[Dict[str, Any]]:
    """
    Semantic search: Finds most results in the vector DB.

    Args:
        query: a question about game industry. 

    Returns:
        A list of dictionaries. Each element contains:
        - Platform: like Game Boy, Playstation 5, Xbox 360...)
        - Name: Name of the Game
        - YearOfRelease: Year when that game was released for that platform
        - Description: Additional details about the game
    """
    
    # Query the collection
    results = collection.query(
        query_texts=[query],
        n_results=5, # Retrieve top 5 similar documents
        include=['metadatas', 'documents']
    )
    
    formatted_results = []
    
    # Parse the results safely
    if results and results.get('metadatas') and results.get('documents'):
        # The query returns a list of lists (batch processing), we take the first index [0]
        for metadata, document in zip(results['metadatas'][0], results['documents'][0]):
            formatted_results.append({
                "Platform": metadata.get("Platform", "Unknown"),
                "Name": metadata.get("Name", "Unknown"),
                "YearOfRelease": metadata.get("YearOfRelease", "Unknown"),
                "Description": document 
            })
            
    return formatted_results

#### Evaluate Retrieval Tool

In [ ]:
# TODO: Create evaluate_retrieval tool
# You might use an LLM as judge in this tool to evaluate the performance
# You need to prompt that LLM with something like:
# "Your task is to evaluate if the documents are enough to respond the query. "
# "Give a detailed explanation, so it's possible to take an action to accept it or not."
# Use EvaluationReport to parse the result
# Tool Docstring:
#    Based on the user's question and on the list of retrieved documents, 
#    it will analyze the usability of the documents to respond to that question. 
#    args: 
#    - question: original question from user
#    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database
#    The result includes:
#    - useful: whether the documents are useful to answer the question
#    - description: description about the evaluation result

class EvaluationReport(BaseModel):
    useful: bool = Field(description="whether the documents are useful to answer the question")
    description: str = Field(description="description about the evaluation result")

@tool
def evaluate_retrieval(question: str, retrieved_docs:Optional[List[Dict[str, Any]]] = None) -> EvaluationReport:
    """
    Based on the user's question and on the list of retrieved documents, 
    it will analyze the usability of the documents to respond to that question. 
    
    Args: 
    - question: original question from user
    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database
    
    Returns:
    - EvaluationReport: Includes boolean 'useful' and a 'description'.
    """
    
    # Initialize the LLM for evaluation
    # We instantiate a fresh LLM here to act as the independent 'Judge'
    eval_llm = LLM(model="gpt-4o")

    # Format the docs into a string for the prompt
    docs_content = json.dumps(retrieved_docs, indent=2)

    system_prompt = """Your task is to evaluate if the documents are enough to respond the query.
    Give a detailed explanation, so it's possible to take an action to accept it or not."""

    user_message = f"""
    User Question: {question}

    Retrieved Documents:
    {docs_content}
    """

    # Generate the evaluation using the LLM and parse it into the Report format
    # We assume the custom LLM class supports an 'output_schema' or similar argument for structured outputs
    evaluation = eval_llm.invoke(
        system=system_prompt,
        input=user_message,
        output_schema=EvaluationReport
    )
    
    return evaluation

#### Game Web Search Tool

In [5]:
# TODO: Create game_web_search tool
# Please use Tavily client to search the web
# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - question: a question about game industry. 

tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))

@tool
def game_web_search(question: str) -> str:
    """
    Search the web for current or external information about the game industry.
    Use this when the internal database lacks information or for recent news.

    Args:
        question: a specific question about the game industry. 
        
    Returns:
        A summarized string of relevant web search results.
    """
    
    # Perform the search
    response = tavily_client.search(
        query=question,
        search_depth="advanced", 
        include_images=False,
        max_results=5,
        include_raw_content=False,
        include_answer=True # Tries to get a direct answer generated by Tavily
    )
    
    # Format the output
    search_results = ""
    
    # Check for a direct answer first
    answer = response.get("answer")
    if answer:
        search_results += f"Web Search Answer: {answer}\n\n"
    else:
        search_results += "Web Search Results:\n"
        
    # Append specific snippets for context
    for result in response.get("results", []):
        search_results += f"- Title: {result.get('title', 'No Title')}\n"
        search_results += f"  Snippet: {result.get('content', 'No Content')}\n"
        search_results += f"  Source: {result.get('url', 'No URL')}\n"
        search_results += "---\n"
        
    return search_results

In [13]:
AGENT_INSTRUCTIONS = """
You are UdaPlay, an AI Research Agent specializing in the video game industry. 
Your goal is to answer user questions accurately using the available tools.

**Your workflow must be:**
1. **Initial Check:** For any question about games, history, or platforms, first use the `retrieve_game` tool to check internal knowledge.
2. **Evaluation:** Use the `evaluate_retrieval` tool to check if the retrieved documents are sufficient to answer the question.
3. **Action:**
    - If the documents are marked as `useful: true`, synthesize the final answer ONLY from those documents.
    - If the documents are marked as `useful: false` or if the question is about recent news or external topics not found in the DB, proceed to use the `game_web_search` tool.
4. **Final Answer:** Always provide a clear, concise, and direct answer to the user's question based on the most reliable source (VectorDB or Web Search).
5. **Memory:** Maintain context from previous turns to answer follow-up questions effectively.
"""

udaplay_agent = Agent(
    model_name="gpt-4o",
    instructions=AGENT_INSTRUCTIONS,
    tools=[retrieve_game, evaluate_retrieval, game_web_search],
)

### Agent

In [ ]:
# TODO: Create your Agent abstraction using StateMachine
# Equip with an appropriate model
# Craft a good set of instructions 
# Plug all Tools you developed

# TODO: Create your Agent abstraction using StateMachine


# Define the Agent's behavior
AGENT_INSTRUCTIONS = """
You are UdaPlay, an AI Research Agent specializing in the video game industry. 
Your goal is to answer user questions accurately using the available tools.

**Your workflow must be:**
1. **Initial Check:** For any question about games, history, or platforms, first use the `retrieve_game` tool to check internal knowledge.
2. **Evaluation:** Use the `evaluate_retrieval` tool to check if the retrieved documents are sufficient to answer the question.
3. **Action:**
    - If the documents are marked as `useful: true`, synthesize the final answer ONLY from those documents.
    - If the documents are marked as `useful: false` or if the question is about recent news or external topics not found in the DB, proceed to use the `game_web_search` tool.
4. **Final Answer:** Always provide a clear, concise, and direct answer to the user's question based on the most reliable source (VectorDB or Web Search).
5. **Memory:** Maintain context from previous turns to answer follow-up questions effectively.
"""

# Equip the agent with the necessary model and tools
udaplay_agent = Agent(
    llm=LLM(model="gpt-4o"), # Use a powerful model for complex reasoning and tool use
    instructions=AGENT_INSTRUCTIONS,
    tools=[retrieve_game, evaluate_retrieval, game_web_search],
)

TypeError: Agent.__init__() got an unexpected keyword argument 'llm'

In [14]:
# TODO: Invoke your agent
# - When Pokémon Gold and Silver was released?
# - Which one was the first 3D platformer Mario game?
# - Was Mortal Kombat X realeased for Playstation 5?

questions = [
    "When Pokémon Gold and Silver was released?",          
    "Which one was the first 3D platformer Mario game?", 
    "Was Mortal Kombat X released for Playstation 5?"
]

print(f"Testing UdaPlay Agent with {len(questions)} queries...\n")

for q in questions:
    print(f"--- USER: {q} ---")
    
    run_object = udaplay_agent.invoke(q)
    
    final_state = run_object.get_final_state()
    final_response = final_state["messages"][-1].content
    
    print(f"AGENT: {final_response}\n")

Testing UdaPlay Agent with 3 queries...

--- USER: When Pokémon Gold and Silver was released? ---
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor


TypeError: evaluate_retrieval() missing 1 required positional argument: 'retrieved_docs'

### (Optional) Advanced

In [ ]:
# TODO: Update your agent with long-term memory
# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes